In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,8 hours 39 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,"7 days, 12 hours and 46 minutes"
H2O_cluster_name:,H2O_from_python_kevin_0sjooy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,489.0 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
train = pd.read_csv('input/train.csv', index_col=0)
pred = pd.read_csv('input/predict.csv', index_col=0)

In [3]:
train.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,1.12,Premium,H,SI1,61.6,59.0,6.67,6.63,4.10,5363
1,1.14,Very Good,E,SI2,60.0,54.0,6.74,6.97,4.11,5593
2,0.90,Very Good,D,SI2,60.3,63.0,6.12,6.22,3.72,3534
3,0.71,Ideal,E,VS1,61.9,54.0,5.74,5.76,3.56,3212
4,0.34,Very Good,F,SI2,60.0,62.0,4.51,4.55,2.72,447


In [5]:
train = train[['carat','cut','color','clarity','x','y','z','price']]

In [6]:
h2train = h2o.H2OFrame(train)
h2pred = h2o.H2OFrame(pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
x = list(train.drop(columns = 'price').columns)
y = 'price'

In [9]:
automl = H2OAutoML(max_models = 50, seed = 1, max_runtime_secs = 2400, sort_metric = 'RMSE')
automl.train(x,y,training_frame=h2train)

AutoML progress: |█████████████████████████████████████████████Failed polling AutoML progress log: Unexpected HTTP error: ('Connection aborted.', BadStatusLine('GET /3/Jobs/$03017f00000132d4ffffffff$_8be9a16f6370b009cb2da16bddf4de9 HTTP/1.1\r\n'))
███████████ (cancelled) 100%


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_bc134e699dbcd229260348d7ee664333> was cancelled by the user.

In [ ]:
leader_board = automl.leaderboard
leader_board.head()